# 01PAO25-25 - Python, Data Types

![Instituto Superior Tecnológico Quito](Recurso-26.png)

**Nombre:** Germán Del Río  
**Fecha:** 21/07/2055  

---

![Python Logo](python_logo.png)

[-- Enlace al Repositorio](https://github.com/Saquis/MachineLearning/tree/main/Deberes )

# Regresión Logística: Detección de SPAM

En este ejercicio se muestran los fundamentos de la Regresión Logística planteando uno de los primeros problemas que fueron solucionados mediante el uso de técnicas de Machine Learning: la detección de SPAM.

Se propone la construcción de un sistema de aprendizaje automático capaz de predecir si un correo determinado se corresponde con un correo de SPAM o no, para ello, se utilizará el siguiente conjunto de datos:

2007 TREC Public Spam Corpus
The corpus trec07p contains 75,419 messages:

25220 ham
50199 spam
These messages constitute all the messages delivered to a particular server between these dates:

Sun, 8 Apr 2007 13:07:21 -0400
Fri, 6 Jul 2007 07:04:53 -0400

In [4]:
# Instalación de librerías externas
!pip install scikit-learn
!pip install nltk

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 16.4 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

   ---------------------------------------- 0/4 [tqdm]
   ---------------------------------------- 0/4 [tqdm]
   ---------------------------------------- 0/4 [tqdm]
   ---------- ----------------------------- 1/4 [regex]
   -------------------- ------------------- 2/4 [click]
   -------------------- ------------------- 2/4 [click]
   ------------------------------ --------- 3/4 [nltk]
   ------------------------------ --------- 3/4 [nltk]
   ------------------------------ --------- 3/4 [nltk]
   ------------------------------ --------- 3/4 [nltk]
   ------------------------------ --------- 3/4 [nltk]
   ------------------------------ --------- 3/4 [nltk]
   ------------------------------ --------- 3/4 [nltk]
   ---------------------

In [5]:
### 1. Funciones complementarias

En este caso práctico relacionado con la detección de correos electrónicos de SPAM, el conjunto de datos que disponemos esta formado por correos electrónicos, con sus correspondientes cabeceras y campos adicionales. Por lo tanto, requieren un preprocesamiento previo a que sean ingeridos por el algoritmo de Machine Learning.

In [7]:
# Esta clase facilita el preprocesamiento de correos electrónicos que poseen código HTML
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return ''.join(self.fed)

In [8]:
# Esta función se encarga de elimar los tags HTML que se encuentren en el texto del correo electrónico
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [9]:
# Ejemplo de eliminación de los tags HTML de un texto
t = '<tr><td align="left"><a href="../../issues/51/16.html#article">Phrack World News</a></td>'
strip_tags(t)

'Phrack World News'

Además de eliminar los posibles tags HTML que se encuentren en el correo electrónico, deben realizarse otras acciones de preprocesamiento para evitar que los mensajes contengan ruido innecesario. Entre ellas se encuentra la eliminación de los signos de puntuación, eliminación de posibles campos del correo electrónico que no son relevantes o eliminación de los afijos de una palabra manteniendo únicamente la raiz de la misma (Stemming). La clase que se muestra a continuación realiza estas transformaciones.

In [21]:
import email
import string
import nltk
import re

# Asegurarte de tener los stopwords
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

def strip_tags(html_text):
    """Remove HTML tags from text."""
    if not html_text:
        return ""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', str(html_text))

# clase
class Parser:
    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)
    
    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors='ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)
    
    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg['Subject'] else []
        body = self.get_email_body(msg.get_payload(),
                                   msg.get_content_type())
        content_type = msg.get_content_type()
        # Returning the content of the email
        return {"subject": subject,
                "body": body,
                "content_type": content_type}
    
    def get_email_body(self, payload, content_type):
        """Extract the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                            p.get_content_type())
        return body
    
    def tokenize(self, text):
        """Transform a text string in tokens. Perform two main actions,
        clean the punctuation symbols and do stemming of the text."""
        if not text:  # Manejo de texto vacío o None
            return []
            
        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split(" ")))
        # Stemming of the tokens
        return [self.stemmer.stem(w.lower()) for w in tokens if w.lower() not in self.stopwords]

In [22]:
# Prueba rápida
parser = Parser()
test_text = "Hello world! This is a test email."
tokens = parser.tokenize(test_text)
print("Tokens:", tokens)

Tokens: ['hello', 'world', 'test', 'email']


In [24]:
import os

DATASET_PATH = os.path.join("datasets", "trec07p")

def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range(n_elements):
        mail = index[i].split(" ../")
        label = mail[0]
        path = mail[1][:-1]
        path_mail = path.split("/")[-1]
        ret_indexes.append({"label":label, "email_path":os.path.join(DATASET_PATH, os.path.join("data", path_mail))})
    return ret_indexes

In [34]:
import os

# Verificar si la carpeta res existe
if os.path.exists("res"):
    print("✓ La carpeta 'res' existe")
    print("Contenido de res/:")
    for item in os.listdir("res"):
        print(f"  - {item}")
        
    # Verificar si existe trec07p
    if os.path.exists("res/trec07p"):
        print("\n✓ La carpeta 'res/trec07p' existe")
        print("Contenido de res/trec07p/:")
        for item in os.listdir("res/trec07p"):
            print(f"  - {item}")
            
        # Verificar si existe data
        if os.path.exists("res/trec07p/data"):
            print("\n✓ La carpeta 'res/trec07p/data' existe")
            print("Contenido de res/trec07p/data/ (primeros 10 archivos):")
            files = os.listdir("res/trec07p/data")
            for i, item in enumerate(files[:10]):
                print(f"  - {item}")
            if len(files) > 10:
                print(f"  ... y {len(files) - 10} archivos más")
        else:
            print("\n✗ La carpeta 'res/trec07p/data' NO existe")
    else:
        print("\n✗ La carpeta 'res/trec07p' NO existe")
else:
    print("✗ La carpeta 'res' NO existe")
    print("Contenido del directorio actual:")
    for item in os.listdir("."):
        print(f"  - {item}")

✗ La carpeta 'res' NO existe
Contenido del directorio actual:
  - .ipynb_checkpoints
  - deberdos.ipynb
  - debertres.ipynb
  - deberuno.ipynb
  - pandasdeber.ipynb
  - python_logo.png
  - Recurso-26.png
  - Untitled.ipynb
